# LXML

LXML과 Open API를 활용하여 판례를 로컬폴더에 저장합니다.

In [ ]:
!pip install requests
!pip install lxml

In [ ]:
from lxml_OpenAPI import all_in_one as do

In [ ]:
do('손해배상', 3000, 'case_name', '사건명')
do('손해배상', 3000, 'case_summary', '판시사항')
do('손해배상', 3000, 'case_judgement', '판결요지')
do('손해배상', 3000, 'case_content', '판례내용')

# Sentencepiece

parameter로 Vocab size를 받아서 LXML로 저장한 판례 데이터를 학습합니다.

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece_on_project as sop

In [ ]:
sop.train_sp(20000)

In [ ]:
sentencepiece = sop.get_sentencepiece()

# TF-IDF

TF-IDF 알고리즘을 이용하여 로컬파일들을 학습시킵니다.

In [ ]:
# scikit-learn 패키지를 이용하여 tfidf를 구현합니다.
!pip install -U scikit-learn

### sklearn을 이용하여 tfidf를 구현하는 예시입니다.

In [ ]:
# 예시용 import 입니다.
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
test_data = [
'[1] 일반육체노동을 하는 사람 또는 육체노동을 주로 생계활동으로 하는 사람의 가동연한을 경험칙상 만 65세까지로 보아야 하는지 여부(원칙적 적극)',
'[2] 일실수입 산정의 기초가 되는 피해자의 가동연한을 인정하는 기준',
]

In [ ]:
test_data[0]

In [ ]:
# 한글과 공백을 제외하고 모두 제거
s = test_data[0].replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
print(s)

In [ ]:
# 불용어 제거를 위해 불용어 리스트를 작성합니다.
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n']

In [ ]:
X_train=[]
for sentence in test_data:
    temp_X = sentencepiece.EncodeAsPieces(sentence) # 학습된 sentenpiece 모델을 이용합니다.
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
X_train

In [ ]:
temp = ' '.join(X_train[0])
print(temp)
train_test=re.sub('▁', '', temp)
train_test=re.sub('  ', ' ', train_test)
print(train_test)

### 프로젝트상 실제 코드입니다.

In [ ]:
import tfidf
import sys
import os
import re

In [ ]:
# 불용어 제거를 위해 불용어 리스트를 작성합니다.
stopwords=['_', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n']

In [ ]:
def get_files(path):
    file_list = os.listdir(path)
    for file in file_list:
        if 'total' in file:
            file_list.remove(file)
    temp=[]
    for f in file_list:
        temp.append(int(f[:-4]))
    temp = sorted(temp)
    file_list=[]
    
    for i in range(len(temp)):
        file_list.append(str(temp[i])+'.txt')
    
    files = []
    
    for i in range(len(file_list)):
        with open(path + '\\' + file_list[i], 'r') as data: 
            files.append(data.read())
    return file_list, files

In [ ]:
path = os.path.join(os.getcwd(), 'case_name')

file_list, files = get_files(path)

In [ ]:
files


In [ ]:
X_train=[]
for file in files[:500]:
    temp_X = sentencepiece.EncodeAsPieces(file) # 학습된 sentenpiece 모델을 이용합니다.
    
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
#     print(temp_X)
    X_train.append(temp_X)
# print(X_train)

In [ ]:
for file in files[500:1000]:
    temp_X = sentencepiece.EncodeAsPieces(file) # 학습된 sentenpiece 모델을 이용합니다.
    
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
#     print(temp_X)
    X_train.append(temp_X)
# print(X_train)

In [ ]:
for file in files[1000:1500]:
    temp_X = sentencepiece.EncodeAsPieces(file) # 학습된 sentenpiece 모델을 이용합니다.
    
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
#     print(temp_X)
    X_train.append(temp_X)
# print(X_train)

In [ ]:
for file in files[1500:]:
    temp_X = sentencepiece.EncodeAsPieces(file) # 학습된 sentenpiece 모델을 이용합니다.
    
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
#     print(temp_X)
    X_train.append(temp_X)
# print(X_train)

In [ ]:
realdata = []
for i in range(len(X_train)):
    temp = ' '.join(X_train[i])
    temp = re.sub('▁', '', temp)
    realdata.append(temp)

In [ ]:
realdata[0]

In [ ]:
# 앞서 train시키고 불러온 sentencepiece model을 이용하여 데이터를 전처리합니다.
# jupyter notebook 에서 메모리 초과 오류가 흔히 발생하여 위의 예시처럼 500개씩 나눠서 전처리를 진행합니다.
# data = tfidf.preprocessing(sentencepiece, files, stopwords, True)

In [ ]:
# 전처리된 데이터를 이용하여 tfidf를 모델을 학습시킵니다.
model = tfidf.train(realdata)

In [ ]:
# 학습된 tfidf모델을 저장합니다.
tfidf.save(model, 'judgement_tfidf_model')

In [ ]:
# 이미 학습된 tfidf모델을 가져옵니다.
# tfidf.save함수로 저장된 파일의 경우 .pk 확장자로 저장됨을 유의합니다.
model = tfidf.load('tfidf_model.pk')

# tfidf의 결과를 판례일련번호로 json파일로 저장합니다.

In [1]:
import os
import json 
import numpy as np
import codecs
import tfidf

In [2]:
json_path = "tfidf_results" ## your path variable

In [3]:
def get_files(path):
    file_list = os.listdir(path)
    for file in file_list:
        if 'total' in file:
            file_list.remove(file)
    temp=[]
    for f in file_list:
        temp.append(int(f[:-4]))
    temp = sorted(temp)
    file_list=[]
    
    for i in range(len(temp)):
        file_list.append(str(temp[i])+'.txt')
    
    files = []
    
    for i in range(len(file_list)):
        with open(path + '\\' + file_list[i], 'r') as data: 
            files.append(data.read())
    return file_list, files

In [4]:
# json으로 변환하기 위한 python dict의 틀을 잡습니다.
def init_dict(_from, _to):
    results = {}

    path = os.path.join(os.getcwd(), 'case_name')
    file_list, files = get_files(path)

    for idx in range(_from, _to):
        id = file_list[idx][:-4]
        results[id] = {}
        results[id]['precedent'] = {}
        results[id]['precedent']['id'] = id
        results[id]['summary'] = {}
        results[id]['summary']['id'] = id
        results[id]['judgement'] = {}
        results[id]['judgement']['id'] = id
        results[id]['content'] = {}
        results[id]['content']['id'] = id
    return results

In [5]:
def set_tfidf_data(dict_on, DB_diagram, train_from, train_to):
    # DB_diagram 내용을 tfidf를 돌린후
    # DB의 다이어그램에 채울 데이터를 json으로 만들기 위해 dict에 저장합니다.
    # 먼저 미리 저장해놓은 summary용 tfidf모델을 가져옵니다.
    model = tfidf.load(DB_diagram + '_tfidf_model.pk')
    path = os.path.join(os.getcwd(), 'case_' + DB_diagram)
    file_list, files = get_files(path)
    list_temp = file_list[train_from:train_to]
    file_temp = files[train_from:train_to]
    key_weights = tfidf.do(model, file_temp)


    for idx in range(len(file_temp)):
        id = list_temp[idx][:-4]
        temp = []
        for jdx in range(0, key_weights.shape[1]):
            temp.append(jdx)
        dict_on[id][DB_diagram]['key_weight'] = key_weights[idx]            
        dict_on[id][DB_diagram]['key_index'] = np.array(temp)


In [6]:
# DB의 precedent 다이어그램에 채울 데이터를 json으로 만들기 위해 dict에 저장합니다.
def insert_name(results, a, b):
    path = os.path.join(os.getcwd(), 'case_name')
    file_list, files = get_files(path)

    for idx in range(a, b):
        results[file_list[idx][:-4]]['precedent']['casename'] = files[idx][:-1]
    return results

In [7]:
# dict에 data를 넣습니다.
def sss(a, b):
    results = init_dict(a, b)
    results = insert_name(results, a, b)
    set_tfidf_data(results, 'summary', a, b)
    set_tfidf_data(results, 'judgement', a, b)
    set_tfidf_data(results, 'content', a, b)
    return results

In [8]:
case_ID, trash = get_files(os.path.join(os.getcwd(), 'case_name'))

In [9]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [13]:
a = 1800
b = a + 51
results = sss(a, b)
dir_path = os.path.join(os.getcwd(), 'tfidf_results\\')
for i in range(a, b):
    # 앞전에서 받아놨던 일련번호를 개행문자를 제거하고 파일이름에 넣어줍니다.
    file_path = os.path.join(dir_path, case_ID[i][:-4] + '.json')
    with codecs.open(file_path, 'w+', encoding='utf-8') as json_file:
        json.dump(results[case_ID[i][:-4]], json_file, separators=(',', ':'), 
                  sort_keys=True, indent=4, cls=NumpyEncoder)
    print(case_ID[i][:-4] + 'succeed')

207140succeed
207485succeed
207486succeed
207490succeed
207529succeed
207541succeed
207579succeed
207580succeed
207581succeed
207588succeed
207594succeed
208415succeed
208726succeed
208733succeed
208734succeed
208744succeed
208745succeed
208795succeed
208800succeed
208848succeed
208849succeed
209131succeed
209150succeed
209160succeed
209217succeed
209252succeed
209581succeed
209587succeed
209687succeed
209693succeed
209711succeed
209715succeed
209727succeed
209729succeed
209731succeed
209747succeed
209751succeed
209785succeed
209793succeed
209859succeed
210335succeed
210415succeed
210419succeed
210423succeed
210569succeed
210767succeed
210813succeed
210827succeed
210829succeed
210853succeed
210895succeed


## Jupyter Notebook에서 tfdif 분석결과를 확인하는 방법입니다.

In [ ]:
i=0
j=0
for result in results:
    for tfidf in result:
        if tfidf != 0:
            print (str(i) +" "+ str(j) + " "+ str(tfidf) )
        j+=1
    i+=1